# Padding and Scaling Images and their Labels
[Roboflow's Tips for Resizing and Padding](https://blog.roboflow.com/you-might-be-resizing-your-images-incorrectly/)

In [ ]:
import os
# Define the directory path
folder = 'CAR_VS_BOS_001'
directory = '/Users/eric/Desktop/2-Career/Projects/ObjectDetection/hockey/letterbox_and_train_valid_split/labels'

# Get the list of files in the directory
files = sorted(os.listdir(directory))

for file in files:
    file_number = file.split('_')[-1].split('.')[0]
    file_name = file.replace(file_number, f'{str(0) * (5 - len(str(file_number)))}{file_number}').replace('frame_', f'{folder}-')
    # print(os.path.join(directory, file), os.path.join(directory, file_name))
    os.rename(os.path.join(directory, file), os.path.join(directory, file_name))

In [ ]:
import cv2
import os
import numpy as np
import tqdm
from letterbox_and_train_valid_split.image_multiprocessing import parallel_process
from IPython.display import Image, display

def letterbox_image(image, padded_image_size=(2000, 2000)):
    image_height, image_width = image.shape[:2]
    padded_image_height, padded_image_width = padded_image_size
    # Check if the image is smaller than the padded size
    if image_width < padded_image_width and image_height < padded_image_height:
        # Image is smaller, place it randomly in the padded image
        delta_w = padded_image_width - image_width
        delta_h = padded_image_height - image_height
        top_pad = np.random.randint(0, delta_h)
        bottom_pad = delta_h - top_pad
        left_pad = np.random.randint(0, delta_w)
        right_pad = delta_w - left_pad
        scale = 1.0  # No scaling for smaller images
    else:
        # Image is larger, scale it down
        scale = min(padded_image_width / image_width, padded_image_height / image_height)
        new_width = int(image_width * scale)
        new_height = int(image_height * scale)
        image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
        delta_w = padded_image_width - new_width
        delta_h = padded_image_height - new_height
        top_pad = delta_h // 2
        bottom_pad = delta_h - top_pad
        left_pad = delta_w // 2
        right_pad = delta_w - left_pad
    # Apply padding
    color = [0, 0, 0]  # Black padding
    padded_image = cv2.copyMakeBorder(image, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=color)

    return padded_image, scale, (left_pad, right_pad, top_pad, bottom_pad)

def adjust_boxes(file_path, image_shape, scale, padding):
    with open(file_path, 'r') as file:
        boxes = [line.strip().split() for line in file.readlines()]
    left_pad, right_pad, top_pad, bottom_pad = padding
    adjusted_boxes = []
    classificatons = []
    for classificaton, cx, cy, w, h in boxes:
        # Convert from normalized to image coordinates
        cx, cy, w, h = [float(val) for val in [cx, cy, w, h]]
        cx = cx * image_shape[1]  # Convert to original image width
        cy = cy * image_shape[0]  # Convert to original image height
        w = w * image_shape[1]
        h = h * image_shape[0]
        # Scale the boxes
        cx = cx * scale + left_pad
        cy = cy * scale + top_pad
        w = w * scale
        h = h * scale
        x = int(cx - w / 2)
        y = int(cy - h / 2)
        width = int(w)
        height = int(h)
        adjusted_boxes.append((x, y, width, height))
        classificatons.append(classificaton)
    return classificatons, adjusted_boxes


def draw_boxes_on_image(image, boxes):
    for x, y, w, h in boxes:
        top_left = (x, y)
        bottom_right = (x + w, y + h)
        image = cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
    return image


def normalize_boxes(boxes, image_shape):
    normalized_boxes = []
    if len(boxes) > 0:
        image_height, image_width, _ = image_shape
        for x, y, w, h in boxes:
            # Convert corner coordinates to center coordinates
            cx = x + w / 2
            cy = y + h / 2
            # Normalize coordinates round to 5 decimal places
            nx = round(cx / image_width, 5)
            ny = round(cy / image_height, 5)
            nw = round(w / image_width, 5)
            nh = round(h / image_height, 5)
            normalized_boxes.append((nx, ny, nw, nh))
    return normalized_boxes


def process_directory(input_directory, output_directory, target_size=(2000, 2000), dev=False, n=None):
    target_width, target_height = target_size
    output_images_directory = os.path.join(output_directory, f'images')
    output_labels_directory = os.path.join(output_directory, f'labels')
    annotated_images_directory = os.path.join(output_directory, f'annotated_images')

    if not os.path.exists(output_images_directory):
        os.makedirs(output_images_directory)
    if not os.path.exists(output_labels_directory):
        os.makedirs(output_labels_directory)
    if not os.path.exists(annotated_images_directory):
        os.makedirs(annotated_images_directory)

    image_dir = os.path.join(input_directory, 'images')
    label_dir = os.path.join(input_directory, 'labels')

    # iterate through all files in the input directory and display the progress bar
    file_list = sorted(os.listdir(image_dir))
    # file_list = sorted(os.listdir(image_dir))[-5:]
    if n is not None:
        # sample n random files
        file_list = np.random.choice(file_list, n, replace=False)
    for filename in tqdm.tqdm(file_list, desc=f"Processing Images {target_width}x{target_height}"):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_dir, filename)
            image = cv2.imread(image_path)
            # every image that begins with '1699', scale it down have a width no greater than a random value between 3 and 15% of the target width
            if filename.startswith('1699'):
                scale = min(np.random.uniform(0.03, 0.10) * target_width / image.shape[1], 1.0)
                image = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
            padded_image, scale, padding = letterbox_image(image, target_size)

            # Save the padded image
            output_image_path = os.path.join(output_images_directory, filename)
            cv2.imwrite(output_image_path, padded_image)

            # Adjust the bounding box labels
            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_path = os.path.join(label_dir, label_filename)
            if os.path.exists(label_path) and label_path.endswith('.txt'):
                try:
                    classifications, adjusted_boxes = adjust_boxes(label_path, image.shape, scale, padding)
                    annotated_image = draw_boxes_on_image(padded_image, adjusted_boxes)

                    # Save the annotated image
                    annotated_image_path = os.path.join(annotated_images_directory, filename)
                    cv2.imwrite(annotated_image_path, annotated_image)

                    # Save the adjusted bounding box labels
                    output_label_path = os.path.join(output_labels_directory, label_filename)
                    normalized_boxes = normalize_boxes(adjusted_boxes, padded_image.shape)
                    with open(output_label_path, 'w') as f:
                        for classification, normalized_box in zip(classifications, normalized_boxes):
                            f.write(f"{classification} {' '.join([str(x) for x in normalized_box])}" + "\n")
                except UnicodeDecodeError as e:
                    print(f"Error reading file {label_path}: {e}")
                    continue  # Skip this file
        if dev:
            print(filename)
            print('Image shape:', image.shape)
            print('Padded image shape:', padded_image.shape)
            print('Scale:', scale)
            print('Adjusted boxes:', adjusted_boxes)
            print('Normalized boxes:', normalized_boxes)
            display(Image(filename=annotated_image_path))


dev = False
parallel_process = False
input_directory = '/Users/eric/Desktop/2-Career/Projects/ObjectDetection/hockey/letterbox_and_train_valid_split'
n_processes = 4  # Adjust based on your machine's capabilities

if dev:
    n = 3
    target_widths, target_heights = zip(*[(x, x) for x in range(700, 800 + 1, 64)])
    # target_widths, target_heights = zip(*[(x, x) for x in range(128, 3840 + 1, 64)])
    for target_width, target_height in zip(target_widths, target_heights):
        output_directory = f'{input_directory}/dev/{target_width}x{target_height}'
        os.makedirs(output_directory, exist_ok=True)
        target_size = (target_width, target_height)
        process_directory(input_directory, output_directory, target_size=(target_width, target_height), dev=dev, n=n)

imgsz = int(input('Enter image size (e.g. 640): '))
if imgsz != '':
    print(f'Image size: {imgsz}')
    output_directory = f'{input_directory}/{imgsz}x{imgsz}'
    os.makedirs(output_directory, exist_ok=True)
    target_size = (imgsz, imgsz)
    if parallel_process:
        parallel_process(input_directory, output_directory, target_size, n_processes)
    else:
        process_directory(input_directory, output_directory, target_size, dev=dev)




In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import os

def split_dataset(image_dir, annotation_dir, output_directory, train_ratio=0.8):
    images = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
    annotations = sorted(os.listdir(annotation_dir))

    # Ensure corresponding annotation files exist
    images_with_annotations = []
    annotations_filtered = []
    for image in images:
        annotation = image.rsplit('.', 1)[0] + '.txt'
        if annotation in annotations:
            images_with_annotations.append(image)
            annotations_filtered.append(annotation)

    # Split into train and valid sets
    train_images, valid_images, train_annotations, valid_annotations = train_test_split(
        images_with_annotations, annotations_filtered, train_size=train_ratio
    )

    # Function to copy files to a target directory
    def copy_files(files, source_dir, target_dir):
        for file in files:
            shutil.copy(os.path.join(source_dir, file), os.path.join(target_dir, file))

    # Create directories and copy files
    os.makedirs(f'{output_directory}/train/images', exist_ok=True)
    os.makedirs(f'{output_directory}/train/labels', exist_ok=True)
    os.makedirs(f'{output_directory}/valid/images', exist_ok=True)
    os.makedirs(f'{output_directory}/valid/labels', exist_ok=True)

    copy_files(train_images, image_dir, f'{output_directory}/train/images')
    copy_files(valid_images, image_dir, f'{output_directory}/valid/images')
    copy_files(train_annotations, annotation_dir, f'{output_directory}/train/labels')
    copy_files(valid_annotations, annotation_dir, f'{output_directory}/valid/labels')

    if overwrite_yolo_dataset:
        path = '/Users/eric/Desktop/2-Career/Projects/ObjectDetection/hockey/dataset/'
        # overwrite the directories if they exist
        os.makedirs(f'{path}/train/images', exist_ok=True)
        os.makedirs(f'{path}/train/labels', exist_ok=True)
        os.makedirs(f'{path}/valid/images', exist_ok=True)
        os.makedirs(f'{path}/valid/labels', exist_ok=True)

        copy_files(train_images, image_dir, f'{path}/train/images')
        copy_files(valid_images, image_dir, f'{path}/valid/images')
        copy_files(train_annotations, annotation_dir, f'{path}/train/labels')
        copy_files(valid_annotations, annotation_dir, f'{path}/valid/labels')

overwrite_yolo_dataset = True
images_dir = f'{output_directory}/images'
annotations_dir = f'{output_directory}/labels'
split_dataset(images_dir, annotations_dir, output_directory, train_ratio=0.8)
